<a href="https://colab.research.google.com/github/ejrtks1020/machine-learning-basic/blob/main/_13_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommender_System).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 19.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619420 sha256=76c86115ce344bb4f5bbd306112fba26a475caaea4b75df4a0397d1918549888
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


간단한 surprise 실습

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt = False)
data.raw_ratings[:10]

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

id : 196 사용자가 242 라는 item에대해 평점을 3점을 줌

In [ ]:
model = SVD()

In [ ]:
cross_validate(model, data, measures= ['rmse', 'mae'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9317  0.9334  0.9409  0.9326  0.9375  0.9352  0.0035  
MAE (testset)     0.7350  0.7374  0.7396  0.7354  0.7386  0.7372  0.0018  
Fit time          4.67    4.72    4.68    4.67    4.67    4.68    0.02    
Test time         0.16    0.23    0.22    0.15    0.22    0.19    0.04    


{'fit_time': (4.674195766448975,
  4.716702699661255,
  4.680781126022339,
  4.665648698806763,
  4.669947385787964),
 'test_mae': array([0.735037  , 0.73740176, 0.73962931, 0.73537754, 0.73856501]),
 'test_rmse': array([0.93172208, 0.93335432, 0.94089224, 0.93261389, 0.93753337]),
 'test_time': (0.1562497615814209,
  0.23165011405944824,
  0.21817684173583984,
  0.14690017700195312,
  0.22143197059631348)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [ ]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [ ]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype= int)

0부터 데이터가 시작하도록 조절

In [ ]:
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

인접행렬 크기

In [ ]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users +1, n_movies +1)
shape

(943, 1682)

In [ ]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = 1.

adj_matrix 

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1, -1, []
for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    similarity = np.dot(my_vector, user_vector)
    if similarity > best_match:
      best_match = similarity
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 183, Best Match ID: 275


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)


[272, 273, 275, 280, 281, 283, 287, 288, 289, 290, 292, 293, 297, 299, 300, 301, 302, 306, 312, 314, 315, 316, 317, 321, 322, 323, 324, 327, 330, 331, 332, 333, 339, 342, 345, 346, 353, 354, 355, 356, 357, 363, 364, 365, 366, 372, 374, 378, 379, 381, 382, 383, 384, 385, 386, 387, 390, 391, 392, 394, 395, 396, 398, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 412, 414, 416, 417, 418, 419, 420, 422, 424, 425, 426, 427, 428, 430, 431, 432, 435, 442, 446, 447, 448, 449, 450, 451, 452, 454, 455, 457, 460, 461, 462, 468, 469, 470, 471, 472, 473, 474, 478, 495, 500, 507, 517, 522, 525, 530, 539, 540, 543, 545, 546, 548, 549, 550, 551, 553, 557, 558, 560, 561, 562, 563, 565, 566, 567, 568, 570, 571, 574, 575, 576, 577, 580, 581, 582, 585, 587, 589, 590, 594, 596, 602, 623, 626, 627, 630, 633, 635, 639, 646, 648, 651, 652, 654, 657, 664, 668, 671, 677, 678, 681, 683, 684, 685, 690, 691, 692, 695, 696, 708, 709, 714, 718, 719, 720, 724, 726, 727, 731, 733, 734, 736, 738, 741, 742, 745,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []
for user_id, user_vector in enumerate(adj_matrix):

  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 14.832396974191326, Best Match ID: 737


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[297, 312, 317, 342, 356, 366, 379, 384, 392, 402, 404, 407, 417, 422, 428, 433, 448, 454, 469, 473, 495, 510, 516, 526, 527, 549, 567, 602, 635, 649, 650, 654, 658, 661, 664, 696, 731, 746, 750, 754, 915, 918, 925, 929, 950, 968, 1015, 1046]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [ ]:
def compute_cos_similarity(v1, v2):
  norm1 = np.sqrt(np.sum(np.square(v1)))
  norm2 = np.sqrt(np.sum(np.square(v2)))
  dot = np.dot(v1, v2)
  return dot / (norm1 * norm2)

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector =-1, -1, []
for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_simularity = compute_cos_similarity(my_vector, user_vector)
    if cos_simularity > best_match:
      best_match = cos_simularity
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.5278586163659506, Best Match ID: 915


In [ ]:
recommend_list = []
for i, log in enumerate(zip(my_vector, best_match_vector)):
  log1, log2 = log
  if log1 < 1. and log2 > 0.: #다른 유저는 봤지만 나는 안본 영화
    recommend_list.append(i)
print(recommend_list)

[272, 275, 279, 280, 283, 285, 289, 294, 297, 316, 317, 355, 365, 366, 368, 379, 380, 381, 384, 386, 392, 398, 401, 404, 416, 420, 422, 424, 426, 427, 430, 432, 450, 460, 461, 466, 469, 471, 473, 474, 475, 479, 482, 483, 497, 505, 508, 510, 511, 522, 526, 527, 529, 530, 534, 536, 540, 545, 548, 549, 556, 557, 558, 560, 565, 567, 568, 569, 577, 580, 581, 582, 592, 596, 630, 635, 639, 641, 649, 651, 654, 673, 677, 678, 683, 684, 692, 696, 701, 703, 707, 708, 709, 712, 714, 719, 720, 726, 731, 734, 736, 738, 740, 745, 747, 754, 755, 761, 762, 763, 766, 780, 789, 791, 805, 819, 823, 824, 830, 843, 862, 865, 918, 929, 930, 938, 942, 943, 947, 958, 959, 960, 970, 977, 1004, 1008, 1009, 1010, 1013, 1041, 1045, 1069, 1072, 1073, 1078, 1097, 1100, 1108, 1112, 1118, 1134, 1193, 1205, 1207, 1216, 1219, 1267, 1334, 1400, 1427, 1596, 1681]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [ ]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999, -1, []
for user_id, user_vector in enumerate(adj_matrix):

  if my_id != user_id:
    euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
    if euclidean_dist < best_match:
      best_match = euclidean_dist
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 55.06359959174482, Best Match ID: 737


In [ ]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector =-1, -1, []
for user_id, user_vector in enumerate(adj_matrix):
  if my_id != user_id:
    cos_simularity = compute_cos_similarity(my_vector, user_vector)
    if cos_simularity > best_match:
      best_match = cos_simularity
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.569065731527988, Best Match ID: 915


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [ ]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate


In [ ]:
data = Dataset.load_builtin('ml-100k', prompt=False)

* KNN을 사용한 협업 필터링

In [ ]:
 model = KNNBasic()
 cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = 4, verbose = True)
 

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9814  0.9774  0.9746  0.9727  0.9854  0.9783  0.0046  
MAE (testset)     0.7745  0.7726  0.7690  0.7694  0.7776  0.7726  0.0032  
Fit time          0.42    0.74    0.62    0.93    0.41    0.62    0.20    
Test time         7.02    9.23    8.14    6.43    4.07    6.98    1.74    


{'fit_time': (0.4216756820678711,
  0.7370917797088623,
  0.6194815635681152,
  0.9322538375854492,
  0.4127507209777832),
 'test_mae': array([0.77448556, 0.77264219, 0.76902585, 0.76938509, 0.77764875]),
 'test_rmse': array([0.98136466, 0.97740367, 0.97462296, 0.97267695, 0.98535746]),
 'test_time': (7.022366046905518,
  9.22951078414917,
  8.137685060501099,
  6.433651685714722,
  4.072360992431641)}

* SVD를 사용한 협업 필터링

In [ ]:
model = SVD()
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = 4, verbose = True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9351  0.9391  0.9402  0.9273  0.9355  0.9355  0.0045  
MAE (testset)     0.7343  0.7399  0.7402  0.7310  0.7409  0.7373  0.0039  
Fit time          11.35   14.44   14.69   12.22   8.02    12.14   2.43    
Test time         0.58    0.58    0.43    0.28    0.19    0.41    0.16    


{'fit_time': (11.351507425308228,
  14.439698696136475,
  14.692129373550415,
  12.219423532485962,
  8.015851736068726),
 'test_mae': array([0.73428858, 0.73990701, 0.74024502, 0.73102219, 0.7408967 ]),
 'test_rmse': array([0.9351248 , 0.93909459, 0.94022504, 0.92733883, 0.93554335]),
 'test_time': (0.5757460594177246,
  0.5811576843261719,
  0.4280698299407959,
  0.27796196937561035,
  0.19257640838623047)}

* NMF를 사용한 협업 필터링

In [ ]:
model = NMF()
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = 4, verbose = True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9584  0.9619  0.9600  0.9674  0.9633  0.9622  0.0031  
MAE (testset)     0.7540  0.7561  0.7532  0.7625  0.7588  0.7569  0.0034  
Fit time          11.56   16.04   15.81   13.01   8.76    13.04   2.73    
Test time         0.49    0.51    0.38    0.23    0.14    0.35    0.14    


{'fit_time': (11.55679178237915,
  16.04265832901001,
  15.810329914093018,
  13.009305000305176,
  8.763279914855957),
 'test_mae': array([0.75396569, 0.75609417, 0.75317263, 0.76247348, 0.75882886]),
 'test_rmse': array([0.95841444, 0.96188407, 0.9599728 , 0.96740758, 0.96330052]),
 'test_time': (0.4914820194244385,
  0.5076742172241211,
  0.3791999816894531,
  0.23329734802246094,
  0.1419084072113037)}

* SVD++를 사용한 협업 필터링

In [ ]:
model = SVDpp()
cross_validate(model, data, measures = ['rmse', 'mae'], cv = 5, n_jobs = 4, verbose = True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9191  0.9158  0.9164  0.9268  0.9221  0.9201  0.0041  
MAE (testset)     0.7213  0.7179  0.7184  0.7277  0.7255  0.7222  0.0038  
Fit time          624.85  634.46  619.51  633.58  167.52  535.98  184.32  
Test time         12.46   9.56    12.23   5.96    3.22    8.68    3.60    


{'fit_time': (624.8518922328949,
  634.458086013794,
  619.5109956264496,
  633.5792768001556,
  167.51546573638916),
 'test_mae': array([0.72134423, 0.71794848, 0.71843208, 0.7277112 , 0.72547525]),
 'test_rmse': array([0.91912878, 0.91583739, 0.91638444, 0.92682742, 0.92213951]),
 'test_time': (12.456313133239746,
  9.557472944259644,
  12.229044914245605,
  5.9612016677856445,
  3.2181286811828613)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [ ]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization
from surprise import Dataset

In [ ]:
data = Dataset.load_builtin('ml-100k',prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)
raw_data[:, 0] -= 1
raw_data[:, 1] -= 1

In [ ]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:, 1])
shape = (n_users + 1, n_movies +1)
shape

(943, 1682)

In [ ]:
raw_data

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       ...,
       [      275,      1089,         1, 874795795],
       [       12,       224,         2, 882399156],
       [       11,       202,         3, 879959583]])

In [ ]:
adj_matrix = np.ndarray(shape, dtype=int)
for user_id, movie_id, rating, time in raw_data:
  adj_matrix[user_id][movie_id] = rating

In [ ]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
U, S, V = randomized_svd(adj_matrix, n_components = 2) 
#U : 사용자 , V : 아이템(영화), S :트리값 벡터 SVD

In [ ]:
print(U)
print(V)
print(S)

[[ 0.06580431  0.00597507]
 [ 0.01402104 -0.04662601]
 [ 0.00565798 -0.02561846]
 ...
 [ 0.00744452 -0.02502129]
 [ 0.02403119  0.0080961 ]
 [ 0.04224209 -0.01092715]]
[[ 9.59509371e-02  3.51795155e-02  1.99288117e-02 ...  3.03747116e-05
   3.31055915e-04  3.16852950e-04]
 [-8.72398147e-02 -7.02507073e-03 -2.86182142e-02 ... -4.48134999e-04
   1.05230265e-04  2.03150753e-04]]
[640.63362257 244.83634567]


In [ ]:
S = np.diag(S)#U : 사용자 , V : 아이템(영화), S :트리값 벡터 SVD

In [ ]:
S

array([[640.63362257,   0.        ],
       [  0.        , 244.83634567]])

In [ ]:
print(U.shape)
print(S.shape)
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [ ]:
np.matmul(np.matmul(U,S) , V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

In [ ]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector =-1, -1, []
for user_id, user_vector in enumerate(U):
  if my_id != user_id:
    cos_simularity = compute_cos_similarity(my_vector, user_vector)
    if cos_simularity > best_match:
      best_match = cos_simularity
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999942295956324, Best Match ID: 235


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.: #다른 유저는 봤지만 나는 안본 영화
    recommend_list.append(i)
print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [ ]:
my_id, my_vector = 0, V.T[0]
best_match, best_match_id, best_match_vector =-1, -1, []
for user_id, user_vector in enumerate(V.T):
  if my_id != user_id:
    cos_simularity = compute_cos_similarity(my_vector, user_vector)
    if cos_simularity > best_match:
      best_match = cos_simularity
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999951364145, Best Match ID: 1287


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [ ]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [ ]:
A, B ,iter = non_negative_factorization(adj_matrix, n_components=2)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


In [ ]:
A, B, iter

(array([[1.18237652, 0.60747176],
        [0.        , 0.79439006],
        [0.        , 0.36970596],
        ...,
        [0.        , 0.39191842],
        [0.41896635, 0.2688732 ],
        [0.73811063, 0.42798489]]),
 array([[1.76920279e+00, 1.10262934e+00, 2.68726075e-01, ...,
         0.00000000e+00, 1.22921604e-02, 1.21895880e-02],
        [2.66544395e+00, 2.98039568e-01, 6.94383810e-01, ...,
         6.00068209e-03, 3.10500324e-05, 0.00000000e+00]]),
 64)

In [ ]:
np.matmul(A,B)

array([[3.71104576e+00, 1.48477367e+00, 7.39553953e-01, ...,
        3.64524488e-03, 1.45528239e-02, 1.44126827e-02],
       [2.11740217e+00, 2.36759669e-01, 5.51611593e-01, ...,
        4.76688218e-03, 2.46658370e-05, 0.00000000e+00],
       [9.85430518e-01, 1.10187005e-01, 2.56717834e-01, ...,
        2.21848794e-03, 1.14793821e-05, 0.00000000e+00],
       ...,
       [1.04463657e+00, 1.16807195e-01, 2.72141802e-01, ...,
        2.35177782e-03, 1.21690795e-05, 0.00000000e+00],
       [1.45790287e+00, 5.42099440e-01, 2.99288379e-01, ...,
        1.61342260e-03, 5.15835007e-03, 5.10702716e-03],
       [2.44663712e+00, 9.41418871e-01, 4.95535351e-01, ...,
        2.56820126e-03, 9.08626324e-03, 8.99726450e-03]])

* 사용자 기반 추천

In [ ]:
my_id, my_vector = 0, U[0]
best_match, best_match_id, best_match_vector =-1, -1, []
for user_id, user_vector in enumerate(A):
  if my_id != user_id:
    cos_simularity = compute_cos_similarity(my_vector, user_vector)
    if cos_simularity > best_match:
      best_match = cos_simularity
      best_match_id = user_id
      best_match_vector = user_vector
print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999999995632288, Best Match ID: 494


In [ ]:
recommend_list = []
for i, log in enumerate(zip(adj_matrix[my_id], adj_matrix[best_match_id])):
  log1, log2 = log
  if log1 < 1. and log2 > 0.:
    recommend_list.append(i)
print(recommend_list)

[281, 287, 356, 377, 378, 379, 384, 385, 388, 390, 391, 392, 394, 401, 402, 403, 412, 415, 416, 417, 418, 420, 422, 430, 431, 432, 434, 440, 443, 446, 447, 448, 450, 451, 464, 469, 471, 477, 478, 490, 495, 497, 500, 503, 504, 506, 510, 520, 522, 549, 558, 565, 567, 572, 574, 575, 576, 577, 580, 581, 589, 615, 621, 628, 630, 632, 635, 636, 641, 649, 654, 657, 659, 661, 664, 670, 673, 678, 683, 704, 731, 738, 741, 767, 769, 789, 795, 796, 830, 842, 923, 943, 968, 1038, 1045, 1078, 1090, 1109, 1115, 1117, 1118, 1132, 1134, 1156, 1181, 1182, 1187, 1206, 1207, 1244, 1262, 1418, 1443, 1468, 1541]


* 항목 기반 추천

In [ ]:
my_id, my_vector = 0, B.T[0]  # 기준 항목이 0번 이라고 가정
best_match, best_match_id, best_match_vector = -1, -1, []

for item_id, item_vector in enumerate(B.T):
  if my_id != item_id:
    cos_similarity = compute_cos_similarity(my_vector, item_vector)
    if cos_similarity > best_match:
      best_match = cos_similarity
      best_match_id = item_id
      best_match_vector = item_vector

print('Best Match: {}, Best Match ID: {}'.format(best_match, best_match_id))

Best Match: 0.9999997862195674, Best Match ID: 255


In [ ]:
recommend_list = []
for i, user_vector in enumerate(adj_matrix):
  if adj_matrix[i][my_id] > 0.9:
    recommend_list.append(i)
print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 